In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime as dt
from datetime import date
import calendar
from calendar import weekday, day_name
import re

In [2]:
key = pd.read_csv('key.csv')
weather = pd.read_csv('weather.csv', parse_dates=['date'])
train = pd.read_csv('train.csv', parse_dates=['date'])

In [3]:
train = pd.merge(train,key)

In [4]:
rt = train[train['units'] > 0].sort_values(by=['units'], ascending=False).reset_index(drop=True)

In [5]:
token = ('+FCFCTSGRRADZSNSGGSPLICFG+FGBRUPHZFUVADUDSPOSOSASSPYSQDRSHFZMIPRBCBLVC')

In [6]:
# codesum 분류를 위한 기준 설정 및 추출 함수
ls = np.array(['+FC', 'FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG+', 
               'FG', 'BR', 'UP', 'HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SO', 'SA', 'SS',
                'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC'])



def validation(token):
    result3 = []
    for each in ls:
        if each in token:
            token = token.replace(each,'')
            result3.append(each)
    return result3


def extract_cs():
    test = list(weather.codesum.unique())
    result = []
    for each in test:
        result.append(validation(each))
    return result


In [7]:
# 위 codesum 추출 함수를 기반으로 codesum value에 해당하는 컬럼 및 값 부여
cs_iter = (set(validation(token)) for x in weather.codesum)
cs = sorted(set.union(*cs_iter))

dummies = pd.DataFrame(np.zeros((len(weather), len(cs))), columns = cs)


for i, gen in enumerate(weather.codesum):
    dummies.ix[i, validation(gen)] = '1'
    
weather_cs = weather.join(dummies)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [8]:
weather_cs.DZ.value_counts()

0.0    20020
1        497
Name: DZ, dtype: int64

In [9]:
df = pd.merge(rt, weather_cs, on=['date', 'station_nbr'])

In [10]:
def get_holidays(fpath):
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.split(" ")[:3] for line in lines]
    lines = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines = pd.to_datetime(lines)
    return pd.DataFrame({"date2":lines})

def get_holiday_names(fpath):
    # holiday_names are holidays + around Black Fridays
    
    f = open(fpath)
    lines = f.readlines()
    lines = [line.strip().split(" ")[:4] for line in lines]
    lines_dt = ["{} {} {}".format(line[0], line[1], line[2]) for line in lines]
    lines_dt = pd.to_datetime(lines_dt)
    lines_hol = [line[3] for line in lines]
    return pd.DataFrame({"date2":lines_dt, "holiday_name":lines_hol})

def preprocess(_df, is_train):
    
    df = _df.copy()
    
    # date
    df['date2'] = pd.to_datetime(df['date'])
    
    # weekday
    df['weekday'] = df.date2.dt.weekday
    df['is_weekend'] = df.date2.dt.weekday.isin([5,6])
    df['is_holiday'] = df.date2.isin(holidays.date2)
    df['is_holiday_weekday'] = df.is_holiday & (df.is_weekend == False)
    df['is_holiday_weekend'] = df.is_holiday &  df.is_weekend

    # bool to int (maybe no meaning)
    df.is_weekend = np.where(df.is_weekend, 1, 0)
    df.is_holiday = np.where(df.is_holiday, 1, 0)
    df.is_holiday_weekday = np.where(df.is_holiday_weekday, 1, 0)
    df.is_holiday_weekend = np.where(df.is_holiday_weekend, 1, 0)
    
    # day, month, year
    df['day'] = df.date2.dt.day
    df['month'] = df.date2.dt.month
    df['year'] = df.date2.dt.year
    
    # around BlackFriday
    df = pd.merge(df, holiday_names, on='date2', how = 'left')
    df.loc[df.holiday_name.isnull(), "holiday_name"] = ""

    around_BlackFriday = ["BlackFridayM3", "BlackFridayM2", "ThanksgivingDay", "BlackFriday",
                          "BlackFriday1", "BlackFriday2", "BlackFriday3"]
    df["around_BlackFriday"] = np.where(df.holiday_name.isin(around_BlackFriday), 
                                        df.holiday_name, "Else")

    return df

In [11]:
holidays = get_holidays("walmart_holidays.txt")
holiday_names = get_holiday_names("walmart_holiday_names.txt")

In [12]:
df = preprocess(df, True)

In [13]:
# counting M

columns = list(df.columns)

def count_M(columns):
    ls_M_counting = []
    ls_M_columns= []
    ls_all_counting = []
    
    for i in columns:
        a = df[i].value_counts()
        b = df[i].name
        
        if 'M' in a:
            ls_M_counting.append(a['M'])
            ls_M_columns.append(b)
            ls_all_counting.append(a.sum())
    return ls_M_counting, ls_M_columns, ls_all_counting

        
    
k = count_M(columns)
k = np.array(k).T

M_columns = ['M', 'column', 'total']
dfdf = pd.DataFrame(data = k, columns = M_columns)
dfdf['M'] = dfdf['M'].astype(int)
dfdf['total'] = dfdf['total'].astype(int)
dfdf['Pct'] = dfdf['M'] / dfdf['total'] * 100
dfdf.sort_values(by='Pct', ascending=False)

,M,column,total,Pct
3,79710,depart,118696,67.154748
8,42900,snowfall,118696,36.142751
11,4957,sealevel,118696,4.176215
2,4520,tavg,118696,3.808047
6,4520,heat,118696,3.808047
7,4520,cool,118696,3.808047
5,4492,wetbulb,118696,3.784458
0,3018,tmax,118696,2.542630
1,3016,tmin,118696,2.540945
10,2987,stnpressure,118696,2.516513


In [14]:
# M -> numeric

for col in df.columns:
    if df[col].name == 'weekday' or df[col].name == 'codesum' or df[col].name == 'holiday_name' or df[col].name == 'around_BlackFriday': continue
    elif df[col].dtypes == 'object':
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else: continue


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118696 entries, 0 to 118695
Data columns (total 68 columns):
date                  118696 non-null datetime64[ns]
store_nbr             118696 non-null int64
item_nbr              118696 non-null int64
units                 118696 non-null int64
station_nbr           118696 non-null int64
tmax                  115678 non-null float64
tmin                  115680 non-null float64
tavg                  114176 non-null float64
depart                38986 non-null float64
dewpoint              116125 non-null float64
wetbulb               114204 non-null float64
heat                  114176 non-null float64
cool                  114176 non-null float64
sunrise               49541 non-null float64
sunset                49541 non-null float64
codesum               118696 non-null object
snowfall              74553 non-null float64
preciptotal           101316 non-null float64
stnpressure           115709 non-null float64
sealevel             